In [3]:
import pandas as pd
import numpy as np
import os

In [237]:
usecols = ["Chr", "Start", "End", "Ref", "Alt", "Func.refGene", "Otherinfo10", "Otherinfo11"]
dtype = {"Chr":"category",
          "Start":np.int32,
          "End":np.int32,
          "Ref":"category",
          "Alt":"category",
          "Func.refGene":"category",
          "Otherinfo10":"category",
          "Otherinfo11":"string"}

In [435]:
df1_ = pd.read_csv("chr17_22241.csv", usecols=usecols, dtype=dtype, sep='\t')
df1_ = pd.eval("DP = df.Otherinfo11.str.split(';',3).str[2].str.split('DP=').str[1].astype('int32')", target=df)
df1_ = pd.eval("AF = df.Otherinfo11.str.split(';',5).str[4].str.split('AF=').str[1].astype('float')", target=df)

df2_ = pd.read_csv("chr17_22334.csv", usecols=["Chr", "Start", "End", "Ref", "Alt"], sep='\t')
df2_ = df2_.append(df.iloc[-1])

In [438]:
# this works on Chr, Ref, Alt, Func.refGene, Otherinfo10
def category_filters(df, **kwargs):
    for key, value in kwargs.items():
        if key == "FuncrefGene":
            df = df[df["Func.refGene"] == value]
        else:
            df = df[df[key] == value]
    return df

In [439]:
# for Func.refGene, enter 'FuncrefGene' instead 
display(category_filters(df1_,FuncrefGene="exonic", Otherinfo10="PASS"))

,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
106,chr17,410351,410351,G,T,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=133;VD=66;AF=0.496...,133,0.4962
107,chr17,410391,410391,T,C,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=117;VD=65;AF=0.555...,117,0.5556
108,chr17,410444,410444,C,A,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=152;VD=8;AF=0.0526...,152,0.0526
110,chr17,410508,410508,G,A,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=207;VD=98;AF=0.473...,207,0.4734
111,chr17,410632,410632,C,A,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=344;VD=5;AF=0.0145...,344,0.0145
...,...,...,...,...,...,...,...,...,...,...
50362,chr17,82831752,82831752,T,C,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=814;VD=388;AF=0.47...,814,0.4767
50449,chr17,82884148,82884148,A,C,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=382;VD=148;AF=0.38...,382,0.3874
50471,chr17,82905981,82905981,T,C,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=613;VD=311;AF=0.50...,613,0.5073
50472,chr17,82905989,82905989,G,T,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=671;VD=7;AF=0.0104...,671,0.0104


In [442]:
# this assumes greater than i.e 'DP = 15' means keep DP vals > 15
def num_filter(df, **kwargs):
    for key, value in kwargs.items():
        df.loc[:,key].where(df.loc[:,key] > value,inplace=True)
    return df.dropna()

In [443]:
display(num_filter(df1_,DP=150,AF=.15))

,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
0,chr17,137603,137603,C,T,exonic,Q10,SAMPLE=VE_22241;TYPE=SNV;DP=521;VD=287;AF=0.55...,521.0,0.5509
1,chr17,137604,137604,G,A,exonic,Q10,SAMPLE=VE_22241;TYPE=SNV;DP=506;VD=121;AF=0.23...,506.0,0.2391
3,chr17,138814,138814,-,C,intronic,Q10,SAMPLE=VE_22241;TYPE=Insertion;DP=191;VD=33;AF...,191.0,0.1728
4,chr17,138824,138824,C,A,intronic,Q10,SAMPLE=VE_22241;TYPE=SNV;DP=186;VD=60;AF=0.322...,186.0,0.3226
8,chr17,156366,156366,A,G,exonic,Q10,SAMPLE=VE_22241;TYPE=SNV;DP=453;VD=216;AF=0.47...,453.0,0.4768
...,...,...,...,...,...,...,...,...,...,...
50517,chr17,82941513,82941513,G,T,intronic,Q10,SAMPLE=VE_22241;TYPE=SNV;DP=453;VD=233;AF=0.51...,453.0,0.5143
50519,chr17,82957112,82957112,G,T,intronic,Q10,SAMPLE=VE_22241;TYPE=SNV;DP=168;VD=86;AF=0.511...,168.0,0.5119
50593,chr17,83093088,83093088,G,-,intronic,Q10,SAMPLE=VE_22241;TYPE=Deletion;DP=377;VD=174;AF...,377.0,0.4615
50594,chr17,83093237,83093237,C,G,intronic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=520;VD=519;AF=0.99...,520.0,0.9981


In [546]:
def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    return idx - np.abs(value - array[idx-1]) < np.abs(value - array[idx])

In [547]:
def subtraction(df1, df2):
    # joins cols 1 - start -> 4 - alt, concat df2 col at btm of df1 col
    uqid_df1 = df1[df1.columns[1:5]].apply(lambda x: ''.join(x.astype(str)),axis=1).squeeze()
    uqid_df2 = df2[df2.columns[1:5]].apply(lambda x: ''.join(x.astype(str)),axis=1).squeeze()  
    uqid = pd.concat([uqid_df1, uqid_df2], ignore_index=True)
    uqid_df = pd.DataFrame({'idx':np.arange(0, len(uqid.index))}, index=uqid)
    uqid_df.reset_index(inplace=True)
    max_val = uqid_df.idx.iat[-1]
    # index of all unique values
    uqid_df = uqid_df.sort_index().groupby('index').filter(lambda x: len(x) == 1)
    idx_arr = uqid_df['idx'].to_numpy()
    df1 = df1.drop(df1.index[common_vals[:sep_dfs]])
    df2 = df2.drop(df2.index[common_vals[sep_dfs:] - len(uqid_df1.index)])
    return [df1,df2]

In [548]:
sub_dfs = subtraction(df1_, df2_)
sub_df1 = sub_dfs[0]
sub_df2 = sub_dfs[1]
del sub_dfs

<ipython-input-547-8e345186aa50>:14: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  df1 = df1.drop(df1.index[common_vals[:sep_dfs]])
<ipython-input-547-8e345186aa50>:15: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  df2 = df2.drop(df2.index[common_vals[sep_dfs:] - len(uqid_df1.index)])


In [549]:
display(sub_df1)
display(sub_df2)

,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
0,chr17,137603,137603,C,T,exonic,Q10,SAMPLE=VE_22241;TYPE=SNV;DP=521;VD=287;AF=0.55...,521.0,0.5509
1,chr17,137604,137604,G,A,exonic,Q10,SAMPLE=VE_22241;TYPE=SNV;DP=506;VD=121;AF=0.23...,506.0,0.2391
2,chr17,138812,138812,-,G,intronic,Q10,SAMPLE=VE_22241;TYPE=Insertion;DP=194;VD=2;AF=...,194.0,NaN
3,chr17,138814,138814,-,C,intronic,Q10,SAMPLE=VE_22241;TYPE=Insertion;DP=191;VD=33;AF...,191.0,0.1728
4,chr17,138824,138824,C,A,intronic,Q10,SAMPLE=VE_22241;TYPE=SNV;DP=186;VD=60;AF=0.322...,186.0,0.3226
...,...,...,...,...,...,...,...,...,...,...
50596,chr17,83093330,83093330,G,T,intronic,pSTD;Q10,SAMPLE=VE_22241;TYPE=SNV;DP=126;VD=2;AF=0.0159...,NaN,NaN
50597,chr17,83094142,83094142,G,T,intronic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=131;VD=131;AF=1;BI...,NaN,1.0000
50598,chr17,83094172,83094172,G,A,intronic,pSTD;Q10,SAMPLE=VE_22241;TYPE=SNV;DP=184;VD=2;AF=0.0109...,184.0,NaN
50599,chr17,83094661,83094661,C,T,UTR3,p8;pSTD;Q10,SAMPLE=VE_22241;TYPE=SNV;DP=146;VD=2;AF=0.0137...,NaN,NaN


,Chr,Start,End,Ref,Alt,AF,DP,Func.refGene,Otherinfo10,Otherinfo11
2,chr17,137724,137724,G,A,NaN,NaN,NaN,NaN,NaN
3,chr17,138274,138274,G,T,NaN,NaN,NaN,NaN,NaN
4,chr17,138424,138424,C,A,NaN,NaN,NaN,NaN,NaN
5,chr17,138789,138789,G,T,NaN,NaN,NaN,NaN,NaN
8,chr17,143369,143369,G,A,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
58115,chr17,83090506,83090506,A,-,NaN,NaN,NaN,NaN,NaN
58116,chr17,83092103,83092103,C,T,NaN,NaN,NaN,NaN,NaN
58117,chr17,83092946,83092946,C,A,NaN,NaN,NaN,NaN,NaN
58123,chr17,83094157,83094157,G,A,NaN,NaN,NaN,NaN,NaN


In [565]:
x = (df1_.iloc[24385:24495])
y = (df2_.iloc[27128:27238])

uqid_df1 = x[x.columns[1:5]].apply(lambda x: ''.join(x.astype(str)),axis=1).squeeze()
uqid_df2 = y[y.columns[1:5]].apply(lambda x: ''.join(x.astype(str)),axis=1).squeeze()  
uqid = pd.concat([uqid_df1, uqid_df2], ignore_index=True)

uqid_df1_len = len(uqid_df1.index)
uqid_df2_len = len(uqid_df2.index)
idx = np.append(np.arange(0, uqid_df1_len), np.arange(0, uqid_df1_len))
uqid_df = pd.DataFrame({'idx':idx}, index=uqid)

uqid_df.reset_index(inplace=True)
max_val = uqid_df.idx.iat[-1]
uqid_df = uqid_df.sort_index().groupby('index').filter(lambda x: len(x) == 1)

idx = uqid_df['idx'].to_numpy()
print(idx)

# search first 5th to end x[len(x)//5:].argmin()
mid_val = idx[len(idx)//5:].argmin() + len(idx)//5
inv_arr = lambda max_val,idx_arr: np.array(sorted(set(range(0, max_val)).difference(idx_arr)))
print(idx[:mid_val])
x = x.drop(x.index[inv_arr(uqid_df1_len, idx[:mid_val])])
y = y.drop(y.index[inv_arr(uqid_df2_len, idx[mid_val:])])

display(x)
display(y)

[  5   7   9  10  11  19  22  26  32  41  42  46  47  48  49  54  58  59
  63  65  67  69  74  75  76  77  78  80  82  83  84  86  89  94  99 100
 101 102 104 105 106 107 108 109   6   7   9  10  13  19  33  39  43  44
  48  50  51  58  59  60  61  62  64  65  67  69  74  75  76  77  78  81
  86  87  95  96  97  98  99 101 102 103 104 105 106 107 108 109]
[  5   7   9  10  11  19  22  26  32  41  42  46  47  48  49  54  58  59
  63  65  67  69  74  75  76  77  78  80  82  83  84  86  89  94  99 100
 101 102 104 105 106 107 108 109]


,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
24390,chr17,41149549,41149563,GGATTCACAGCAATA,-,exonic,PASS,SAMPLE=VE_22241;TYPE=Deletion;DP=392;VD=4;AF=0...,392.0,NaN
24392,chr17,41160177,41160177,T,C,UTR3,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=594;VD=6;AF=0.0101...,594.0,NaN
24394,chr17,41167233,41167233,C,T,UTR3,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=4;VD=4;AF=1;BIAS=0...,NaN,1.0000
24395,chr17,41167649,41167649,C,A,exonic,p8,SAMPLE=VE_22241;TYPE=SNV;DP=386;VD=4;AF=0.0104...,386.0,NaN
24396,chr17,41167852,41167852,A,G,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=322;VD=5;AF=0.0155...,322.0,NaN
24404,chr17,41177735,41177735,G,T,UTR3,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=250;VD=8;AF=0.032;...,250.0,NaN
24407,chr17,41177958,41177958,A,G,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=747;VD=9;AF=0.012;...,747.0,NaN
24411,chr17,41184205,41184205,A,G,UTR3,pSTD,SAMPLE=VE_22241;TYPE=SNV;DP=145;VD=2;AF=0.0138...,NaN,NaN
24417,chr17,41184572,41184572,G,T,intronic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=137;VD=3;AF=0.0219...,NaN,NaN
24426,chr17,41190111,41190111,C,A,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=564;VD=9;AF=0.016;...,564.0,NaN


,Chr,Start,End,Ref,Alt,AF,DP,Func.refGene,Otherinfo10,Otherinfo11
27134,chr17,41160402,41160402,C,A,NaN,NaN,NaN,NaN,NaN
27135,chr17,41160427,41160427,G,T,NaN,NaN,NaN,NaN,NaN
27137,chr17,41167613,41167613,G,T,NaN,NaN,NaN,NaN,NaN
27138,chr17,41167777,41167777,G,T,NaN,NaN,NaN,NaN,NaN
27141,chr17,41167949,41167949,C,A,NaN,NaN,NaN,NaN,NaN
27147,chr17,41177747,41177747,G,T,NaN,NaN,NaN,NaN,NaN
27161,chr17,41184599,41184603,GGCGG,AACGA,NaN,NaN,NaN,NaN,NaN
27167,chr17,41190021,41190021,C,A,NaN,NaN,NaN,NaN,NaN
27171,chr17,41190342,41190342,-,TGCTGTGGGTCCAGCTGCTGCCAGCCT,NaN,NaN,NaN,NaN,NaN
27172,chr17,41226939,41226939,C,A,NaN,NaN,NaN,NaN,NaN


In [538]:
df1_drop_idxs = invert_arr(idx_arr[:sep_dfs], max_val)
df2_drop_idxs = invert_arr(idx_arr[sep_dfs:], max_val + 1) - len(uqid_df1.index)
print(df1_drop_idxs)
print(df2_drop_idxs)
x = x.drop(x.index[[df1_drop_idxs]])
y = y.drop(y.index[[df2_drop_idxs]])

display(x)
display(y)
# 4114950841149509CTTG
# 4114950841149509CTTG

In [496]:
sub_dfs = subtraction(df1_, df2_)
sub_df1 = sub_dfs[0]
sub_df2 = sub_dfs[1]
del sub_dfs

In [507]:
display(sub_df1.iloc[[24385]])
display(sub_df2.iloc[[27128]])
display(df1_.iloc[[24385]])

,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
24385,chr17,41149508,41149509,CT,TG,exonic,NM5.25,SAMPLE=VE_22241;TYPE=Complex;DP=445;VD=136;AF=...,445.0,0.3056


,Chr,Start,End,Ref,Alt,AF,DP,Func.refGene,Otherinfo10,Otherinfo11
27128,chr17,41149508,41149509,CT,TG,NaN,NaN,NaN,NaN,NaN


,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
24385,chr17,41149508,41149509,CT,TG,exonic,NM5.25,SAMPLE=VE_22241;TYPE=Complex;DP=445;VD=136;AF=...,445.0,0.3056


In [210]:
#TODO: make option to see what is NA and what is not NA
#TODO: make program and functions run on a class
# possible error of pd.to_numeric int out of range may come from long ref/alt strings